### Libraries and source files

In [82]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [83]:
# --  Choose which file to import from below
extended_df_name = 'extended_fmeca.xlsx' # The whole fmeca - which has all the newly calculated columns
req_calc_name = 'fmeca_req_calc.xlsx' # A slice of the fmeca with selected columns that require calculation. The order of columns is important here.
correct_abts_name = 'correct_abts.xlsx' # A slice of the fmeca which requires abt corrections.
correct_dim_name = 'correct_dim.xlsx' # A slice of fmeca which requires correct diminution levels

# -- File path to import from
file_location = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working/ProcessedData'

In [84]:
df = pd.read_excel(os.path.join(file_location,req_calc_name),sheet_name='Sheet1')

In [85]:
df.head(3)

,Stiffener(s),stiff_anom_combo,stiffener_dim,h,grade,ps,Average UTM Reading (mm),tp,tw,tf,"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),Frame(s),Primary Structure,Detail Structure,Final Anomaly description,Location,Detail
0,BP,BP_4SWBT-LBHD-CR-545,350x19.5 + 0x0 L2,0.000,AH36,Sec,14.40,14.40,14.40,14.40,TB,0.0,FR-47,Long'l bhd,NaN,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining
1,LL00,LL00_4SWBT-LBHD-CR-219,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.99,6.99,6.99,6.99,FR-47,0.0,FR-46,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining
2,LL00,LL00_4SWBT-LBHD-CR-220,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.60,6.60,6.60,6.60,FR-47,650.0,FR-46,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining


### Importing as-built thicknesses

In [86]:
dm_file = 'decisionMatrix.xlsx' 
pathANworking = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working'
decisionMatrixPath = os.path.join(pathANworking,dm_file)

In [87]:
abts = pd.read_excel(decisionMatrixPath,sheet_name='abts')

In [88]:
abts.head(2)

,Stiffener,t basePlate,tweb,tflange
0,LL00,14.0,10.0,15.5
1,LL01,14.0,10.0,15.5


In [89]:
a = dict(zip(abts['Stiffener'],abts['t basePlate']))

In [90]:
# -- creating a vlookpup function

def vlookup(df,seek_col,out_col):
    a = dict(zip(abts[seek_col],abts[out_col]))
    return a


### Data cleaning

Renaming parameters, harmonising structure names

#### General modifications

In [91]:
def renameColumns():
    df.rename(columns={"Stiffener(s)":"stiffener","Nearest transverse member (Fr., TWF,TBHD)":"nearest_transverse","Longitudinal\n(l) (distance\nfrom in mm)":"distance_to_transverse","Frame(s)":"frame",
    "Final Anomaly description":"anomaly_type","Average UTM Reading (mm)":"Avg_utm","Primary Structure":"primary_structure","Location":"location",
    "Detail":"detail","Detail Structure":"detail_structure"},inplace=True)
    return None

renameColumns()

In [92]:
df["frame"].unique()

array(['FR-47', 'FR-46', 'FR-49', 'FR-48', 'FR.50-51', 'TWF47-48',
       'TWF54-55', 'TWF48-49', 'TWF50-51', 'TWF54', 'FR 46-47',
       'TWF46-47', '49-50', '52-53'], dtype=object)

In [93]:
existing_map = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'TWF47-48', 'TWF54-55',
       'TWF48-49', 'TWF50-51', 'TWF54', 'TWF46-47', '49-50', '52-53']
new_map = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'FR-48', 'FR-55',
       'FR-49', 'FR-51', 'FR-54', 'FR-47', 'FR-50', 'FR-53']

map_dict = dict(zip(existing_map,new_map))

In [94]:
df["frame"] = df["frame"].map(map_dict)

In [95]:
df[df["nearest_transverse"]=='TB']

,stiffener,stiff_anom_combo,stiffener_dim,h,grade,ps,Avg_utm,tp,tw,tf,nearest_transverse,distance_to_transverse,frame,primary_structure,detail_structure,anomaly_type,location,detail
0,BP,BP_4SWBT-LBHD-CR-545,350x19.5 + 0x0 L2,0.00,AH36,Sec,14.4,14.4,14.4,14.4,TB,0.0,FR-47,Long'l bhd,NaN,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining
176,LL23,LL23_4SWBT-LBHD-CR-450,500x10.5 + 150x23 L2,4.49,AH36,Sec,12.4,12.4,12.4,12.4,TB,530.0,FR-47,Long'l bhd,NaN,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining
177,LL23,LL23_4SWBT-LBHD-CR-452,500x10.5 + 150x23 L2,4.49,AH36,Sec,0.0,0.0,0.0,0.0,TB,210.0,FR-47,Long'l bhd,NaN,Generalised corrosion,Tank boundary plating,Less than 6mm remaining
178,LL23,LL23_4SWBT-LBHD-CR-453,500x10.5 + 150x23 L2,4.49,AH36,Sec,11.4,11.4,11.4,11.4,TB,0.0,FR-47,Long'l bhd,NaN,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining
219,LL27,LL27_4SWBT-LBHD-CR-538,350x10 + 100x14 L2,1.00,AH36,Sec,6.5,6.5,6.5,6.5,TB,530.0,FR-47,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining


#### Changing frames to the nearest defined frame in global loads calculation

Not all the frames are defined in the global loads calculation.<br>
Therefore, the rule that is being followed on this project is to change the frame to the nearest defined frame such that the bending moment of the frame can be applied in the calculations.
In the below script a problem specific solution has been implemented. A more general code can be written to replace it with the nearest defined Frame name.

In [96]:
# -- Existing cleaned frame names
a = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'FR-48', 'FR-55',
       'FR-49', 'FR-51', 'FR-54', 'FR-47', 'FR-50', 'FR-53'] 

# -- Nearest defined frames
b = ['46', '46', '51', '46', '46', '56',
       '51', '51', '56', '46', '51', '51']

map_nearest_frame = dict(zip(a,b))

In [97]:
df['frame_number'] = df['frame'].map(map_nearest_frame)

#### Horizontal location of the anomaly

In reference to the nearest anomlay.
The nearest anomaly column has inconsistencies.
The names in this column will be replaced with proper names.

In [98]:
df['nearest_transverse'].unique()

array(['TB', 'FR-47', 'FR-46', 'TBHD 46', 'FR-48', 'FR-49', 'FR-50',
       'TWF47', 'TWF54', 'TWF48', 'TWF49', 'TWF51', 'Vert Stiffener',
       'TWF46', 'FR.49', 'FR.53', 'Vertical Stiffener', 'LBHD'],
      dtype=object)

In [99]:
old_trns = ['TB', 'FR-47', 'FR-46', 'TBHD 46', 'FR-48', 'FR-49', 'TWF47',
       'TWF54', 'TWF48', 'TWF49', 'TWF51', 'Vert Stiffener', 'TWF46',
       'FR.49', 'FR.53', 'Vertical Stiffener', 'LBHD']
new_trns = ['TB', 'FR-47', 'FR-46', 'FR-46', 'FR-48', 'FR-49', 'FR-47',
       'FR-54', 'FR-48', 'FR-49', 'FR-51', 'Vertical Stiffener', 'FR-46',
       'FR-49', 'FR-53', 'Vertical Stiffener', 'LBHD']

map_nearest_trns = dict(zip(old_trns,new_trns))

In [100]:
df['nearest_transverse'] = df['nearest_transverse'].map(map_nearest_trns)

In [101]:
df['stiff_anom_combo'] = df['stiff_anom_combo'].apply(lambda x: x.replace(' ',''))

In [102]:
df.head(3)

,stiffener,stiff_anom_combo,stiffener_dim,h,grade,ps,Avg_utm,tp,tw,tf,nearest_transverse,distance_to_transverse,frame,primary_structure,detail_structure,anomaly_type,location,detail,frame_number
0,BP,BP_4SWBT-LBHD-CR-545,350x19.5 + 0x0 L2,0.000,AH36,Sec,14.40,14.40,14.40,14.40,TB,0.0,FR-47,Long'l bhd,NaN,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,46
1,LL00,LL00_4SWBT-LBHD-CR-219,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.99,6.99,6.99,6.99,FR-47,0.0,FR-46,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46
2,LL00,LL00_4SWBT-LBHD-CR-220,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.60,6.60,6.60,6.60,FR-47,650.0,FR-46,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46


#### Thickness columns

In [103]:
abts.head(2)

,Stiffener,t basePlate,tweb,tflange
0,LL00,14.0,10.0,15.5
1,LL01,14.0,10.0,15.5


In [104]:
def clean_tp(row):
    abt_dict = vlookup(abts,'Stiffener','t basePlate')
    if row['location'] == 'Tank boundary plating':
        return row['tp']
    else:
        return abt_dict[row['stiffener']]
    
def clean_tw(row):
    abt_dict = vlookup(abts,'Stiffener','tweb')
    if row['detail_structure'] == 'Web':
        return row['tw']
    else:
        return abt_dict[row['stiffener']]
    

def clean_tf(row):
    abt_dict = vlookup(abts,'Stiffener','tflange')
    if row['detail_structure'] == 'Flange' or row['detail_structure'] == "Flat bar stiff'r":
        return row['tf']
    else:
        return abt_dict[row['stiffener']]

In [105]:
df['tp'] = df.apply(clean_tp,axis=1)
df['tw'] = df.apply(clean_tw,axis=1)
df['tf'] = df.apply(clean_tf,axis=1)

In [106]:
df.head()

,stiffener,stiff_anom_combo,stiffener_dim,h,grade,ps,Avg_utm,tp,tw,tf,nearest_transverse,distance_to_transverse,frame,primary_structure,detail_structure,anomaly_type,location,detail,frame_number
0,BP,BP_4SWBT-LBHD-CR-545,350x19.5 + 0x0 L2,0.000,AH36,Sec,14.40,14.4,13.50,25.0,TB,0.0,FR-47,Long'l bhd,NaN,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,46
1,LL00,LL00_4SWBT-LBHD-CR-219,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.99,14.0,6.99,15.5,FR-47,0.0,FR-46,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46
2,LL00,LL00_4SWBT-LBHD-CR-220,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.60,14.0,6.60,15.5,FR-47,650.0,FR-46,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46
3,LL00,LL00_4SWBT-LBHD-CR-221,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.30,14.0,6.30,15.5,FR-47,650.0,FR-46,Long'l bhd,Web,Pit within generalised corrosion,Stiffener web,Greater than 6mm remaining,46
4,LL00,LL00_4SWBT-LBHD-CR-223,350x10 + 150x15.5 L2,23.497,AH36,Sec,9.50,14.0,10.00,9.5,FR-46,2050.0,FR-46,Long'l bhd,Flange,Generalised corrosion,Stiffener flange middle 1/3,Greater than 6mm remaining,46


### Data Munging

#### New column for span

In [107]:
def span(row,span1,span2):
    stiffener = row['stiffener']
    if stiffener == "LL26" or stiffener == "LL27" or stiffener == 'BP' or stiffener == "UD":
        return span2
    else:
        return span1



In [108]:
df['span'] = df.apply(span, args=(5095,2547.5),axis=1)

In [109]:
df.head(3)

,stiffener,stiff_anom_combo,stiffener_dim,h,grade,ps,Avg_utm,tp,tw,tf,nearest_transverse,distance_to_transverse,frame,primary_structure,detail_structure,anomaly_type,location,detail,frame_number,span
0,BP,BP_4SWBT-LBHD-CR-545,350x19.5 + 0x0 L2,0.000,AH36,Sec,14.40,14.4,13.50,25.0,TB,0.0,FR-47,Long'l bhd,NaN,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,46,2547.5
1,LL00,LL00_4SWBT-LBHD-CR-219,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.99,14.0,6.99,15.5,FR-47,0.0,FR-46,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46,5095.0
2,LL00,LL00_4SWBT-LBHD-CR-220,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.60,14.0,6.60,15.5,FR-47,650.0,FR-46,Long'l bhd,Web,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46,5095.0


#### Anomaly position ratio

In [110]:
def positionRatio(row):
    position = row['distance_to_transverse']
    span = row['span']
    try:
        return np.round(position/span,2)
    except:
        return 'error'

In [111]:
df['anom_position_ratio'] = df.apply(positionRatio, axis=1)

In [112]:
df['anom_position_ratio'].unique()

array([0.  , 0.13, 0.4 , 0.23, 0.27, 0.28, 0.39, 0.16, 0.04, 0.35, 0.47,
       0.25, 0.36, 0.03, 0.54, 0.41, 0.26, 0.37, 0.52, 0.18, 0.3 , 0.07,
       0.1 , 0.5 , 0.14, 0.21, 0.22, 0.2 , 0.19, 0.48, 0.11, 0.08, 0.43,
       0.02, 0.38, 0.06, 0.09, 0.6 , 0.76, 0.7 , 0.12, 0.44, 0.33, 0.34,
       0.32, 0.05, 0.17, 0.45, 0.29, 0.24, 0.01, 0.49, 0.51, 0.58, 0.15,
       0.61, 0.31])

#### Bending moment ratio calculation

In [113]:
def bmRatioFixedBeam(row):
    '''Bending moment ratio for a given x location along the stiffener
       Applicable only if the arrangement is Fixed end beam
       '''
    try:
        x = float(row['distance_to_transverse'])
        l = float(row['span'])
        if x != 0:
            try:
                x = float(row['distance_to_transverse'])
                l = float(row['span'])
                bmPerc = np.round(np.abs(((6*l*x)-(l**2)-(6*x**2))/(l**2)),2)
            except:
                bmPerc = 'error'
        else:
            bmPerc = 'anom located at the end'
    except:
        bmPerc = 'error'
    return bmPerc

In [114]:
def sfRatioFixedBeam(row):
    '''Shear force ratio for a given x location along the stiffener
       Applicable only if the arrangement is Fixed end beam
       '''
    try:
        x = float(row['distance_to_transverse'])
        l = float(row['span'])
        if x != 0:
            try:
                x = float(row['distance_to_transverse'])
                l = float(row['span'])
                sfPerc = np.round(np.abs(((l/2)-x)/(l/2)),2)
            except:
                sfPerc = 'error'
        else:
            sfPerc = 'anom located at the end'
    except:
        sfPerc = 'error'
    return sfPerc

In [115]:
df['bm%'] = df.apply(bmRatioFixedBeam,axis=1)

In [116]:
df['sf%'] = df.apply(sfRatioFixedBeam,axis=1)

In [117]:
df[df['stiff_anom_combo']=='LL00_4SWBT-LBHD-CR-606']

,stiffener,stiff_anom_combo,stiffener_dim,h,grade,ps,Avg_utm,tp,tw,tf,nearest_transverse,distance_to_transverse,frame,primary_structure,detail_structure,anomaly_type,location,detail,frame_number,span,anom_position_ratio,bm%,sf%
16,LL00,LL00_4SWBT-LBHD-CR-606,350x10 + 150x15.5 L2,23.497,AH36,Sec,11.1,14.0,10.0,11.1,NaN,0.0,NaN,Long'l bhd,Flange,Generalised corrosion,Stiffener flange middle 1/3,Greater than 6mm remaining,NaN,5095.0,0.0,anom located at the end,anom located at the end


In [118]:
abts

,Stiffener,t basePlate,tweb,tflange
0,LL00,14.0,10.0,15.5
1,LL01,14.0,10.0,15.5
2,LL02,14.0,10.0,17.0
3,LL03,14.0,10.0,17.0
4,LL04,11.5,10.0,18.5
5,LL05,11.5,10.0,18.5
6,LL06,11.5,10.0,17.0
7,LL07,11.5,10.0,17.0
8,LL08,12.0,10.0,20.0
9,LL09,12.0,10.0,20.0


In [119]:
xw.view(df)

#### Compare Anomalies with current calculation

DO NOT use this section unless specific stiffener/anomalies are required to be dropped. <br>

Applicable only if some anomalies have been dropped manually and if the new calc required sheet in the middle of the process.

In [121]:

#-- Batch 5A defined
calc = 'FSC-JAD-01-TEC-62-A1 MV LBHD Calcs AN.xlsm' 
pathBatch5a = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets'
shtcalc = '3-Sect properties'
cellRangeBatch5a = 'A24:AK270'

In [122]:
nameFile = calc
path = pathBatch5a
sheet = shtcalc
cellRange = cellRangeBatch5a

In [123]:
file_path = os.path.join(path,nameFile)
book = xw.Book(file_path)
sht = book.sheets[sheet]
rng = sht.range(cellRange)
calc_df = rng.options(pd.DataFrame, index=False, header=True).value

In [157]:
anomalies = calc_df["Anomaly / Stiff. ID"].tolist()

In [158]:
anomalies = [i for i in anomalies if len(i) > 5]

In [160]:
anom_exist = set(anomalies)

In [161]:
anom_new = set(df["stiff_anom_combo"])

In [169]:
diff = anom_new.difference(anom_exist)

In [173]:
diff1 = [i for i in diff if len(i.split('_')[0]) > 2]

In [174]:
diff1

['LL25_4SWBT-LBHD-CR-494', 'LL25_4SWBT-LBHD-CR-498', 'LL23_4SWBT-LBHD-CR-452']

In [175]:
# -- These items have been dropped previously for various reasons.
# therfore they wont be carried this time.

dropThese = diff1

In [176]:
df_new = df[~df["stiff_anom_combo"].isin(dropThese)]

In [178]:
xw.view(df_new)

In [168]:
len(anom_exist)

218